In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, save_model, load_model

In [ ]:
##### 데이터 병합 과정 #####

In [85]:
#this was created in general/diagnosis_making notebook
diag = pd.read_csv("D:/Desktop/aicoss-model/preprocessed/ground_truth.csv").drop("Unnamed: 0", axis=1)

Below we are combining several clinical datasets.

In [86]:
# PTDEMOG 파일 읽기
demo = pd.read_csv("D:/Desktop/aicoss-model/PTDEMOG.csv").rename(columns = {"PHASE":"Phase"})
demo.columns

Index(['Phase', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'VISDATE', 'PTSOURCE',
       'PTGENDER', 'PTDOB', 'PTDOBYY', 'PTHAND', 'PTMARRY', 'PTEDUCAT',
       'PTWORKHS', 'PTWORK', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTADBEG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT',
       'PTIDENT', 'PTORIENT', 'PTORIENTOT', 'PTENGSPK', 'PTNLANG',
       'PTENGSPKAGE', 'PTCLANG', 'PTLANGSP', 'PTLANGWR', 'PTSPTIM',
       'PTSPOTTIM', 'PTLANGPR1', 'PTLANGSP1', 'PTLANGRD1', 'PTLANGWR1',
       'PTLANGUN1', 'PTLANGPR2', 'PTLANGSP2', 'PTLANGRD2', 'PTLANGWR2',
       'PTLANGUN2', 'PTLANGPR3', 'PTLANGSP3', 'PTLANGRD3', 'PTLANGWR3',
       'PTLANGUN3', 'PTLANGPR4', 'PTLANGSP4', 'PTLANGRD4', 'PTLANGWR4',
       'PTLANGUN4', 'PTLANGPR5', 'PTLANGSP5', 'PTLANGRD5', 'PTLANGWR5',
       'PTLANGUN5', 'PTLANGPR6', 'PTLANGSP6', 'PTLANGRD6', 'PTLANGWR6',
       'PTLANGUN6', 'PTLANGTTL', 'PTETHCATH', 'PTASIAN', 'PTOPI', 'PTBORN',
       'PTBIRPL', 'PTIMMAGE', 'PTIMMWHY', 'PTBIRPR', 'PTBIRGR'

In [87]:
# NEUROEXM 파일 읽기
neuro = pd.read_csv("D:/Desktop/aicoss-model/NEUROEXM.csv").rename(columns = {"PHASE":"Phase"})

In [5]:
neuro.columns

Index(['Phase', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'VISDATE', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'ID', 'SITEID', 'USERDATE', 'USERDATE2',
       'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR',
       'update_stamp'],
      dtype='object')

In [88]:
# ADSP_PHC_COGN 파일 읽기
clinical = pd.read_csv("D:/Desktop/aicoss-model/ADSP_PHC_COGN.csv").rename(columns = {"PHASE":"Phase"})

In [89]:
with pd.option_context('display.max_seq_items', None,
                       'display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', None):
    display(clinical.columns)

Index(['RID', 'PTID', 'SUBJID', 'Phase', 'VISCODE', 'VISCODE2', 'EXAMDATE',
       'PHC_Visit', 'PHC_Age_Cognition', 'PHC_Diagnosis', 'PHC_Sex',
       'PHC_Race', 'PHC_Ethnicity', 'PHC_Education', 'PHC_MEM', 'PHC_MEM_SE',
       'PHC_MEM_PreciseFilter', 'PHC_EXF', 'PHC_EXF_SE',
       'PHC_EXF_PreciseFilter', 'PHC_LAN', 'PHC_LAN_SE',
       'PHC_LAN_PreciseFilter', 'PHC_VSP', 'PHC_VSP_SE',
       'PHC_VSP_PreciseFilter', 'update_stamp'],
      dtype='object')

In [90]:
diag["Subject"].value_counts()

052_S_1352    6
007_S_2394    6
127_S_1427    6
067_S_2304    6
068_S_2187    5
             ..
072_S_4465    1
011_S_4519    1
116_S_4013    1
094_S_4486    1
035_S_4786    1
Name: Subject, Length: 1138, dtype: int64

In [91]:
# DXSUM 파일 읽기
comb = pd.read_csv("D:/Desktop/aicoss-model/DXSUM.csv")[["RID", "PTID", "PHASE"]].rename(columns = {"PHASE":"Phase"})
comb

,RID,PTID,Phase
0,1247,023_S_1247,ADNI1
1,1245,141_S_1245,ADNI1
2,1289,023_S_1289,ADNI1
3,1242,128_S_1242,ADNI1
4,1249,116_S_1249,ADNI1
...,...,...,...
6114,2123,128_S_2123,ADNI4
6115,4598,135_S_4598,ADNI4
6116,4381,037_S_4381,ADNI4
6117,4212,067_S_4212,ADNI4


In [92]:
# PTDEMOG, NEUROEXM, ADSP_PHC_COGN 데이터 병합
m = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"]).merge(clinical,on = ["RID", "Phase"]).drop_duplicates()

d:\Desktop\aicoss-model\env37\lib\site-packages\pandas\core\frame.py:9204: FutureWarning: Passing 'suffixes' which cause duplicate columns {'PTID_x'} in the result is deprecated and will raise a MergeError in a future version.
  validate=validate,


In [93]:
with pd.option_context('display.max_seq_items', None,
                       'display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', None):
    display(m.columns)
    print(m.shape)

Index(['RID', 'PTID_x', 'Phase', 'PTID_y', 'VISCODE_x', 'VISCODE2_x',
       'VISDATE_x', 'PTSOURCE', 'PTGENDER', 'PTDOB', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTWORKHS', 'PTWORK', 'PTNOTRT', 'PTRTYR',
       'PTHOME', 'PTTLANG', 'PTPLANG', 'PTADBEG', 'PTCOGBEG', 'PTADDX',
       'PTETHCAT', 'PTRACCAT', 'PTIDENT', 'PTORIENT', 'PTORIENTOT', 'PTENGSPK',
       'PTNLANG', 'PTENGSPKAGE', 'PTCLANG', 'PTLANGSP', 'PTLANGWR', 'PTSPTIM',
       'PTSPOTTIM', 'PTLANGPR1', 'PTLANGSP1', 'PTLANGRD1', 'PTLANGWR1',
       'PTLANGUN1', 'PTLANGPR2', 'PTLANGSP2', 'PTLANGRD2', 'PTLANGWR2',
       'PTLANGUN2', 'PTLANGPR3', 'PTLANGSP3', 'PTLANGRD3', 'PTLANGWR3',
       'PTLANGUN3', 'PTLANGPR4', 'PTLANGSP4', 'PTLANGRD4', 'PTLANGWR4',
       'PTLANGUN4', 'PTLANGPR5', 'PTLANGSP5', 'PTLANGRD5', 'PTLANGWR5',
       'PTLANGUN5', 'PTLANGPR6', 'PTLANGSP6', 'PTLANGRD6', 'PTLANGWR6',
       'PTLANGUN6', 'PTLANGTTL', 'PTETHCATH', 'PTASIAN', 'PTOPI', 'PTBORN',
       'PTBIRPL', 'PTIMMAGE', 'PTIMMWHY', 'P

(4045, 136)


In [94]:
# merge 작업 후 중복되는 열 제거
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [95]:
with pd.option_context('display.max_seq_items', None,
                       'display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', None):
    display(m.columns)
    print(m.shape)

Index(['RID', 'PTID', 'Phase', 'VISCODE', 'VISCODE2', 'VISDATE', 'PTSOURCE',
       'PTGENDER', 'PTDOB', 'PTDOBYY', 'PTHAND', 'PTMARRY', 'PTEDUCAT',
       'PTWORKHS', 'PTWORK', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTADBEG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT',
       'PTIDENT', 'PTORIENT', 'PTORIENTOT', 'PTENGSPK', 'PTNLANG',
       'PTENGSPKAGE', 'PTCLANG', 'PTLANGSP', 'PTLANGWR', 'PTSPTIM',
       'PTSPOTTIM', 'PTLANGPR1', 'PTLANGSP1', 'PTLANGRD1', 'PTLANGWR1',
       'PTLANGUN1', 'PTLANGPR2', 'PTLANGSP2', 'PTLANGRD2', 'PTLANGWR2',
       'PTLANGUN2', 'PTLANGPR3', 'PTLANGSP3', 'PTLANGRD3', 'PTLANGWR3',
       'PTLANGUN3', 'PTLANGPR4', 'PTLANGSP4', 'PTLANGRD4', 'PTLANGWR4',
       'PTLANGUN4', 'PTLANGPR5', 'PTLANGSP5', 'PTLANGRD5', 'PTLANGWR5',
       'PTLANGUN5', 'PTLANGPR6', 'PTLANGSP6', 'PTLANGRD6', 'PTLANGWR6',
       'PTLANGUN6', 'PTLANGTTL', 'PTETHCATH', 'PTASIAN', 'PTOPI', 'PTBORN',
       'PTBIRPL', 'PTIMMAGE', 'PTIMMWHY', 'PTBIRPR', 'PTBIRGR'

(4045, 119)


In [96]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [97]:
print(diag)

            PTID  Group  Phase
0     023_S_1247    1.0  ADNI1
1     141_S_1245    1.0  ADNI1
2     023_S_1289    2.0  ADNI1
3     128_S_1242    0.0  ADNI1
4     116_S_1249    0.0  ADNI1
...          ...    ...    ...
1976  130_S_4294    2.0  ADNI4
1977  114_S_2392    0.0  ADNI4
1978  067_S_4782    2.0  ADNI4
1979  037_S_4381    0.0  ADNI4
1980  067_S_4212    0.0  ADNI4

[1981 rows x 3 columns]


In [98]:
# ground_truth까지 병합 -> 진단명 라벨링
m = m.merge(diag, on = ["PTID", "Phase"])

In [99]:
m["PTID"].value_counts()

067_S_4767    36
067_S_4072    28
027_S_1387    26
067_S_4054    24
002_S_4262    21
              ..
020_S_4619     1
016_S_4601     1
128_S_4603     1
136_S_4517     1
035_S_4786     1
Name: PTID, Length: 1129, dtype: int64

In [100]:
with pd.option_context('display.max_seq_items', None,
                       'display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', None):
    display(m.columns)
    print(m.shape)

Index(['RID', 'PTID', 'Phase', 'VISCODE', 'VISCODE2', 'VISDATE', 'PTSOURCE',
       'PTGENDER', 'PTDOB', 'PTDOBYY', 'PTHAND', 'PTMARRY', 'PTEDUCAT',
       'PTWORKHS', 'PTWORK', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTADBEG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT',
       'PTIDENT', 'PTORIENT', 'PTORIENTOT', 'PTENGSPK', 'PTNLANG',
       'PTENGSPKAGE', 'PTCLANG', 'PTLANGSP', 'PTLANGWR', 'PTSPTIM',
       'PTSPOTTIM', 'PTLANGPR1', 'PTLANGSP1', 'PTLANGRD1', 'PTLANGWR1',
       'PTLANGUN1', 'PTLANGPR2', 'PTLANGSP2', 'PTLANGRD2', 'PTLANGWR2',
       'PTLANGUN2', 'PTLANGPR3', 'PTLANGSP3', 'PTLANGRD3', 'PTLANGWR3',
       'PTLANGUN3', 'PTLANGPR4', 'PTLANGSP4', 'PTLANGRD4', 'PTLANGWR4',
       'PTLANGUN4', 'PTLANGPR5', 'PTLANGSP5', 'PTLANGRD5', 'PTLANGWR5',
       'PTLANGUN5', 'PTLANGPR6', 'PTLANGSP6', 'PTLANGRD6', 'PTLANGWR6',
       'PTLANGUN6', 'PTLANGTTL', 'PTETHCATH', 'PTASIAN', 'PTOPI', 'PTBORN',
       'PTBIRPL', 'PTIMMAGE', 'PTIMMWHY', 'PTBIRPR', 'PTBIRGR'

(5066, 120)


In [ ]:
##### 데이터 가공 #####

In [165]:
t = m

In [ ]:
# 데이터 병합용으로 사용했던 feature drop
t.columns
t = t.drop(["ID", "SITEID", "USERDATE", "USERDATE2", "update_stamp", "PTSOURCE", 'PHC_Diagnosis'], axis=1) 
# t = t.drop(["VISCODE", "VISCODE2"], axis=1)

In [167]:
with pd.option_context('display.max_seq_items', None,
                       'display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', None):
    display(t.columns)

Index(['RID', 'PTID', 'Phase', 'VISCODE', 'VISCODE2', 'VISDATE', 'PTGENDER',
       'PTDOB', 'PTDOBYY', 'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTWORKHS',
       'PTWORK', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTTLANG', 'PTPLANG',
       'PTADBEG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'PTIDENT',
       'PTORIENT', 'PTORIENTOT', 'PTENGSPK', 'PTNLANG', 'PTENGSPKAGE',
       'PTCLANG', 'PTLANGSP', 'PTLANGWR', 'PTSPTIM', 'PTSPOTTIM', 'PTLANGPR1',
       'PTLANGSP1', 'PTLANGRD1', 'PTLANGWR1', 'PTLANGUN1', 'PTLANGPR2',
       'PTLANGSP2', 'PTLANGRD2', 'PTLANGWR2', 'PTLANGUN2', 'PTLANGPR3',
       'PTLANGSP3', 'PTLANGRD3', 'PTLANGWR3', 'PTLANGUN3', 'PTLANGPR4',
       'PTLANGSP4', 'PTLANGRD4', 'PTLANGWR4', 'PTLANGUN4', 'PTLANGPR5',
       'PTLANGSP5', 'PTLANGRD5', 'PTLANGWR5', 'PTLANGUN5', 'PTLANGPR6',
       'PTLANGSP6', 'PTLANGRD6', 'PTLANGWR6', 'PTLANGUN6', 'PTLANGTTL',
       'PTETHCATH', 'PTASIAN', 'PTOPI', 'PTBORN', 'PTBIRPL', 'PTIMMAGE',
       'PTIMMWHY', 'PTBIRPR', 'PTBIRGR', 'DD_CRF_VE

In [168]:
# 결측치가 70% 이상인 feature drop
t = t.fillna(-4)
cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]

# 테스트 버전에서는 실제 drop 하지 않으나 실제 학습시에는 drop
t.drop(cols_to_delete, axis = 1, inplace = True)

In [169]:
with pd.option_context('display.max_seq_items', None,
                       'display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', None):
    display(t.columns)

Index(['RID', 'PTID', 'Phase', 'VISCODE', 'VISCODE2', 'VISDATE', 'PTGENDER',
       'PTDOB', 'PTDOBYY', 'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTNOTRT',
       'PTRTYR', 'PTHOME', 'PTTLANG', 'PTPLANG', 'PTCOGBEG', 'PTADDX',
       'PTETHCAT', 'PTRACCAT', 'NXVISUAL', 'NXAUDITO', 'NXTREMOR', 'NXCONSCI',
       'NXNERVE', 'NXMOTOR', 'NXFINGER', 'NXHEEL', 'NXSENSOR', 'NXTENDON',
       'NXPLANTA', 'NXGAIT', 'NXOTHER', 'NXABNORM', 'SUBJID', 'EXAMDATE',
       'PHC_Visit', 'PHC_Age_Cognition', 'PHC_Sex', 'PHC_Race',
       'PHC_Ethnicity', 'PHC_Education', 'PHC_MEM', 'PHC_MEM_SE',
       'PHC_MEM_PreciseFilter', 'PHC_EXF', 'PHC_EXF_SE',
       'PHC_EXF_PreciseFilter', 'PHC_LAN', 'PHC_LAN_SE',
       'PHC_LAN_PreciseFilter', 'PHC_VSP', 'PHC_VSP_SE',
       'PHC_VSP_PreciseFilter', 'Group'],
      dtype='object')

In [42]:
##### One-hot Encoding #####

# 원본 수정한 버전

In [170]:
KEY_COLS    = ["PTID", "RID", "Phase", "Group", "PHC_Visit", "VISCODE", "VISCODE2"]                         # 존재하면 앞으로 고정
DROP_COLS = ["PTADDX", "PTCOGBEG", "PTDOBYY", "PTRTYR"] # 치매 확진 년도, 증상 인지 시작 년도, 태어난 년도, 은퇴 년도
CAT_MAX_UNIQUE = 10   # 이 값 이하의 카디널리티면 범주형으로 간주 (int/float/obj 공통)
KEEP_DUMMY_NA  = True # 결측(여기선 -4→NaN)을 별도 더미로 보관할지 여부
SENTINEL       = -4   # 이전 단계에서 사용한 결측 대체값
TEXT_COLS = []

# 숫자로 판별되는 feature는 numeric 형으로 변환
def to_numeric_safe(s: pd.Series) -> pd.Series:
    out = pd.to_numeric(s, errors="ignore")
    # 숫자가 된 경우에만 그대로 반환 (object로 남은 건 원문 유지)
    return out


# 결측 처리 규칙: -4는 결측으로 취급 (원핫에서 dummy_na로 쓰고, 수치형은 NaN으로)
def mark_na(s: pd.Series) -> pd.Series:
    if pd.api.types.is_numeric_dtype(s):
        return s.replace(SENTINEL, np.nan)
    # object일 때도 -4(숫자)가 들어있을 수 있으니 같이 치환
    return s.replace(SENTINEL, np.nan)

In [171]:
# 2) -4 문자열을 숫자 -4로 통일 (혹시 문자열로 저장된 경우)
t = t.replace(str(SENTINEL), SENTINEL)

for col in t.columns:
    t[col] = to_numeric_safe(t[col])

In [172]:
# 3) 학습에 필요없는 DROP_COL 제거

t = t.drop(columns=DROP_COLS)

In [173]:
# 4) KEY_COLS가 실제로 존재하는지 정제 (순서 유지)
key_cols = [c for c in KEY_COLS if c in t.columns]
key_cols

['PTID', 'RID', 'Phase', 'Group', 'PHC_Visit', 'VISCODE', 'VISCODE2']

In [174]:
# 5) 결측 처리
t = t.apply(mark_na)

In [175]:
# 6) 텍스트(자유서술형) 열 제외 목록 확정
text_cols = [c for c in TEXT_COLS if c in t.columns]
text_cols

[]

In [177]:
# 7) 열 타입/카디널리티 기반으로 범주형/수치형 자동 분류
#    - object형은 기본적으로 범주형
#    - 숫자형 중에서도 unique 값이 적으면 코드형으로 간주하여 범주형에 포함

cat_cols, num_cols = [], []
for col in t.columns:
    if col in key_cols or col in text_cols:
        continue
    s = t[col]
    nunique = s.nunique(dropna=True)

# 아래 코드는 object형, numeric형 feature의 값의 종류가 CAT_MAX_UNIQUE 이하일 경우 범주형으로 전환하는 코드
# 테스트 과정에서는 무조건 범주형으로 바뀔 것이므로 비활성화한 상태
# 실제 학습 과정에서는 주석 풀고 활성화 시킬 것

    if s.dtype == "object":
        # 장문 텍스트 위험 → TEXT_COLS에 없고 유니크가 너무 크면 제외 권장
        # (원핫 폭발 방지) → 여기서는 CAT_MAX_UNIQUE 초과 시 자동으로 수치/제외 쪽으로 빼지 않고, 그냥 제외.
        if nunique <= CAT_MAX_UNIQUE:
            cat_cols.append(col)
        else:
            # 고카디널리티 object는 원핫 제외 (필요시 hashing trick/target encoding 고려)
            pass
    else:
        # 숫자형: low-card면 코드형(범주형), 아니면 수치형
        if nunique <= CAT_MAX_UNIQUE:
            cat_cols.append(col)
        else:
            num_cols.append(col)


In [178]:
# cat_cols 확정 후: 존재/중복 정리
# cat_cols = [c for c in cat_cols if c in t.columns]
# cat_cols = list(dict.fromkeys(cat_cols))

# 숫자라도 범주로 취급되도록 dtype을 category로 변환
t[cat_cols] = t[cat_cols].apply(lambda s: s.astype('category'))

In [179]:
cat_cols

['PTGENDER',
 'PTHAND',
 'PTMARRY',
 'PTNOTRT',
 'PTHOME',
 'PTTLANG',
 'PTPLANG',
 'PTETHCAT',
 'PTRACCAT',
 'NXVISUAL',
 'NXAUDITO',
 'NXTREMOR',
 'NXCONSCI',
 'NXNERVE',
 'NXMOTOR',
 'NXFINGER',
 'NXHEEL',
 'NXSENSOR',
 'NXTENDON',
 'NXPLANTA',
 'NXGAIT',
 'NXOTHER',
 'NXABNORM',
 'PHC_Sex',
 'PHC_Race',
 'PHC_Ethnicity',
 'PHC_MEM_PreciseFilter',
 'PHC_EXF_PreciseFilter',
 'PHC_LAN_PreciseFilter',
 'PHC_VSP_PreciseFilter']

In [180]:
# 8) 원핫 인코딩 (uint8로 메모리 절감, dummy_na로 결측 더미 보존 옵션)
#    - 열 이름 충돌 방지 위해 prefix=열이름 통일
cat_df = pd.get_dummies(
    t[cat_cols],
    dtype="uint8",
    dummy_na=KEEP_DUMMY_NA
) if cat_cols else pd.DataFrame(index=t.index)

In [181]:
with pd.option_context('display.max_seq_items', None,
                       'display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', None):
    display(cat_df.columns)

Index(['PTGENDER_1.0', 'PTGENDER_2.0', 'PTGENDER_nan', 'PTHAND_1.0',
       'PTHAND_2.0', 'PTHAND_nan', 'PTMARRY_1.0', 'PTMARRY_2.0', 'PTMARRY_3.0',
       'PTMARRY_4.0', 'PTMARRY_5.0', 'PTMARRY_nan', 'PTNOTRT_0.0',
       'PTNOTRT_1.0', 'PTNOTRT_2.0', 'PTNOTRT_nan', 'PTHOME_1.0', 'PTHOME_2.0',
       'PTHOME_3.0', 'PTHOME_4.0', 'PTHOME_5.0', 'PTHOME_6.0', 'PTHOME_7.0',
       'PTHOME_8.0', 'PTHOME_nan', 'PTTLANG_1.0', 'PTTLANG_2.0', 'PTTLANG_nan',
       'PTPLANG_1.0', 'PTPLANG_2.0', 'PTPLANG_3.0', 'PTPLANG_nan',
       'PTETHCAT_1.0', 'PTETHCAT_2.0', 'PTETHCAT_3.0', 'PTETHCAT_nan',
       'PTRACCAT_1.0', 'PTRACCAT_2.0', 'PTRACCAT_3.0', 'PTRACCAT_4.0',
       'PTRACCAT_5.0', 'PTRACCAT_6.0', 'PTRACCAT_7.0', 'PTRACCAT_nan',
       'NXVISUAL_1.0', 'NXVISUAL_2.0', 'NXVISUAL_nan', 'NXAUDITO_1.0',
       'NXAUDITO_2.0', 'NXAUDITO_nan', 'NXTREMOR_1.0', 'NXTREMOR_2.0',
       'NXTREMOR_nan', 'NXCONSCI_1.0', 'NXCONSCI_2.0', 'NXCONSCI_nan',
       'NXNERVE_1.0', 'NXNERVE_2.0', 'NXNERVE_nan', 'N

In [ ]:
# 9) 수치형 처리 (결측은 NaN 유지)
num_df = t[num_cols].copy()
for col in num_cols:
    if not pd.api.types.is_numeric_dtype(num_df[col]):
        num_df[col] = pd.to_numeric(num_df[col], errors="coerce")
num_df.fillna()
# 스케일러 초기화
scaler = MinMaxScaler()

# 수치형 컬럼을 열 단위로 0~1 정규화
num_df_norm = pd.DataFrame(
    scaler.fit_transform(num_df),   # ndarray 반환 → DataFrame으로 감싸줌
    columns=num_df.columns,         # 원래 컬럼명 유지
    index=num_df.index              # 원래 인덱스 유지
)

In [183]:
with pd.option_context('display.max_seq_items', None,
                       'display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', None):
    print(num_cols)
    print(cat_cols)

['PTEDUCAT', 'PHC_Age_Cognition', 'PHC_Education', 'PHC_MEM', 'PHC_MEM_SE', 'PHC_EXF', 'PHC_EXF_SE', 'PHC_LAN', 'PHC_LAN_SE', 'PHC_VSP', 'PHC_VSP_SE']
['PTGENDER', 'PTHAND', 'PTMARRY', 'PTNOTRT', 'PTHOME', 'PTTLANG', 'PTPLANG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL', 'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER', 'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER', 'NXABNORM', 'PHC_Sex', 'PHC_Race', 'PHC_Ethnicity', 'PHC_MEM_PreciseFilter', 'PHC_EXF_PreciseFilter', 'PHC_LAN_PreciseFilter', 'PHC_VSP_PreciseFilter']


In [184]:
# 10) 결과 결합: [KEY] + 원핫 + 수치
out_parts = []
if key_cols: out_parts.append(t[key_cols])
if not cat_df.empty: out_parts.append(cat_df)
if not num_df.empty: out_parts.append(num_df)

out = pd.concat(out_parts, axis=1)

In [185]:
with pd.option_context('display.max_seq_items', None,
                       'display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', None):
    display(out.columns)

Index(['PTID', 'RID', 'Phase', 'Group', 'PHC_Visit', 'VISCODE', 'VISCODE2',
       'PTGENDER_1.0', 'PTGENDER_2.0', 'PTGENDER_nan', 'PTHAND_1.0',
       'PTHAND_2.0', 'PTHAND_nan', 'PTMARRY_1.0', 'PTMARRY_2.0', 'PTMARRY_3.0',
       'PTMARRY_4.0', 'PTMARRY_5.0', 'PTMARRY_nan', 'PTNOTRT_0.0',
       'PTNOTRT_1.0', 'PTNOTRT_2.0', 'PTNOTRT_nan', 'PTHOME_1.0', 'PTHOME_2.0',
       'PTHOME_3.0', 'PTHOME_4.0', 'PTHOME_5.0', 'PTHOME_6.0', 'PTHOME_7.0',
       'PTHOME_8.0', 'PTHOME_nan', 'PTTLANG_1.0', 'PTTLANG_2.0', 'PTTLANG_nan',
       'PTPLANG_1.0', 'PTPLANG_2.0', 'PTPLANG_3.0', 'PTPLANG_nan',
       'PTETHCAT_1.0', 'PTETHCAT_2.0', 'PTETHCAT_3.0', 'PTETHCAT_nan',
       'PTRACCAT_1.0', 'PTRACCAT_2.0', 'PTRACCAT_3.0', 'PTRACCAT_4.0',
       'PTRACCAT_5.0', 'PTRACCAT_6.0', 'PTRACCAT_7.0', 'PTRACCAT_nan',
       'NXVISUAL_1.0', 'NXVISUAL_2.0', 'NXVISUAL_nan', 'NXAUDITO_1.0',
       'NXAUDITO_2.0', 'NXAUDITO_nan', 'NXTREMOR_1.0', 'NXTREMOR_2.0',
       'NXTREMOR_nan', 'NXCONSCI_1.0', 'NXCONSCI_

In [ ]:
# 11) 가장 최근의 진단 결과만 남기기

# 각 환자에 대햐여 가장 최근의 진단 결과만 남긴다
# 각 환자에 대한 과거의 테스트 기록이 없어지게 되는 코드이지만 원본 코드에도 존재하는 부분이었음

# out = out.groupby('PTID', group_keys=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)

out['PHC_Visit'] = pd.to_numeric(out['PHC_Visit'], errors='coerce')
out = out.loc[out.groupby('PTID')['PHC_Visit'].idxmax()].drop(columns=['PHC_Visit']).reset_index(drop=True)
key_cols.remove('PHC_Visit')

In [189]:
# 12) 열 순서 고정(재현성): KEY → cat(사전순) → num(사전순)
ordered_cols = []
ordered_cols += key_cols
ordered_cols += sorted([c for c in out.columns if c not in key_cols and c in cat_df.columns])
ordered_cols += sorted([c for c in out.columns if c not in key_cols and c in num_df.columns])
out = out[ordered_cols]

In [190]:
# 13) csv 파일 생성
out.to_csv("D:/Desktop/aicoss-model/preprocessed/clinical.csv")

### 최종 전처리 결과물: clinical.csv

In [121]:
# 14) train_test_split 및 pkl 파일 생성

# X는 'Group' 열을 제외한 데이터프레임, Y는 'Group' 열
X = out.drop(columns=['Group', 'VISCODE','VISCODE2'])
y = out['Group']

# 데이터 분할 (예: 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train.to_pickle("D:/Desktop/aicoss-model/preprocessed/X_train_c.pkl")
y_train.to_pickle("D:/Desktop/aicoss-model/preprocessed/y_train_c.pkl")

X_test.to_pickle("D:/Desktop/aicoss-model/preprocessed/X_test_c.pkl")
y_test.to_pickle("D:/Desktop/aicoss-model/preprocessed/y_test_c.pkl")